In [ ]:
#Step 1 : Run this cell
import cv2
import numpy as np

In [7]:
def my_histogram(image):
    histogram = np.zeros(256, dtype=int)

    for pixel_value in image.flatten():
        histogram[pixel_value] += 1

    return histogram

In [8]:
def local_histogram_equalization(image, size, stride):
    gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    height, width = gray_image.shape

    result = np.zeros((height, width))

    for i in range(0, height, stride):
        for j in range(0, width, stride):
            local_region = gray_image[i : min(i + size, height), j : min(j + size, width)]
            hist = my_histogram(local_region)
            cdf = hist.cumsum()

            cdf_min = cdf[cdf > 0].min()  # the minimum non-zero value in the CDF

            cdf_normalized = (cdf - cdf_min) * 255 / (cdf[-1] - cdf_min)  # normalizing
            # cdf_normalized = (cdf * 255) / cdf.max()
            equalized_local_region = cdf[local_region].astype(np.uint8)

            result[i : min(i + size, height), j : min(j + size, width)] = equalized_local_region

    return result

In [32]:
#Code to test arrow image in feed
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    height,width,channels  = frame.shape
    #print(height,width)
    if not ret:
        break
    neg_img = 255 - frame
    processed_img = local_histogram_equalization(neg_img,64,32)
    cv2.imshow('Original Image',frame)
    cv2.imshow('Negative Image',neg_img)
    cv2.imshow('Processed Image',processed_img)

    if cv2.waitKey(10) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


In [ ]:
#Step 2 : Run this cell
#Utility Functions for arrow tip detection

#Image preprocessor
def preprocess(img):
    img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    img_blur = cv2.GaussianBlur(img_gray, (5, 5), 1)
    img_canny = cv2.Canny(img_blur, 50, 50)
    kernel = np.ones((3, 3))
    img_dilate = cv2.dilate(img_canny, kernel, iterations=2)
    img_erode = cv2.erode(img_dilate, kernel, iterations=1)
    return img_erode

# Function to match common points between contours and corners
def match_points(contour, good_features, threshold=10):
    matched_points = []
    for pt in good_features:
        for cnt_pt in contour:
            dist = np.linalg.norm(cnt_pt[0] - pt)  # Access the [0] index for the point
            if dist < threshold:
                matched_points.append(cnt_pt[0])  # Store only the coordinates
    return matched_points

def find_tip(points, convex_hull):
    length = len(points)
    indices = np.setdiff1d(range(length), convex_hull)

    for i in range(2):
        j = indices[i] + 2
        if j > length - 1:
            j = length - j
        if np.all(points[j] == points[indices[i - 1] - 2]):
            return tuple(points[j])
# Function to find the tail of the arrow
def find_tail(points, tip):
    # Find the point farthest from the tip
    max_dist = -1
    tail_point = None
    
    for point in points:
        point = tuple(point[0])
        dist = np.linalg.norm(np.array(point) - np.array(tip))
        if dist > max_dist:
            max_dist = dist
            tail_point = point
            
    return tail_point

In [ ]:
#Step 3 : Run this cell
#Iterative template matching function to find the best template & the best scale

methods = [cv2.TM_CCOEFF, cv2.TM_CCOEFF_NORMED, cv2.TM_CCORR,
            cv2.TM_CCORR_NORMED, cv2.TM_SQDIFF, cv2.TM_SQDIFF_NORMED]

#return co-od required to build a rectangle
def iterative_match(frame, template_img_lft,template_img_right):
    MATCH_THRESHOLD = 0.1
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)  # Convert frame to grayscale
    best_value = -1
    best_location = (-1, -1)
    best_scale = -1
    best_template_flag = '' 
    best_template = None

    for scale in np.arange(0.1, 2, 0.01):
        resized_template = 255 - cv2.resize(cv2.imread('right_arrow_template.png', 0), (0, 0), fx=scale, fy=scale)
        resized_template_left = 255 - cv2.resize(cv2.imread('left_arrow_template.png', 0), (0, 0), fx=scale, fy=scale)
        
        # Ensure the template size is smaller than the frame size
        if resized_template.shape[0] > gray_frame.shape[0] or resized_template.shape[1] > gray_frame.shape[1]:
            continue
        
            
        result = cv2.matchTemplate(gray_frame, resized_template, methods[1])  # Using CCOEFF_NORMED
        min_val, max_val, min_loc, max_loc = cv2.minMaxLoc(result)
        left_test_result = cv2.matchTemplate(gray_frame, resized_template_left, methods[1])
        lft_t_min,lft_t_max,lft_t_min_loc,lft_t_max_loc = cv2.minMaxLoc(left_test_result)

        if max_val > best_value and max_val > MATCH_THRESHOLD:  # Use 'and' instead of '&'
            best_value = max_val
            best_location = max_loc
            best_scale = scale
            best_template_flag = 'R'
        
        if lft_t_max > best_value and lft_t_max > MATCH_THRESHOLD:  # Use 'and' instead of '&'
            best_value = lft_t_max
            best_location = lft_t_max_loc
            best_scale = scale
            best_template_flag = 'L'
        
        if best_template_flag == 'R':
            best_template = template_img_right
        elif best_template_flag == 'L':
            best_template = template_img_lft
            
    
    if best_location[0] != -1 and best_location[1] != -1:  # Use 'and' instead of '&'
        h, w = best_template.shape
        w = int(w * best_scale)
        h = int(h * best_scale)

        top_left = best_location
        bottom_right = (top_left[0] + w, top_left[1] + h)
        return top_left, bottom_right, best_template_flag
    return None, None



In [ ]:
#Finally run this
#Main driver function

import numpy as np
import cv2



#Grey scale of template
right_arrow_template = cv2.imread('right_arrow_template.png',0)
left_arrow_template = cv2.imread('left_arrow_template.png',0)

#taking negative of the templates as well
right_arrow_template = 255 - right_arrow_template
left_arrow_template = 255 - left_arrow_template

roi = cv2.imread('right_arrow_template.png',0)
cap = cv2.VideoCapture(0)

feature_params = dict(maxCorners=100, qualityLevel=0.05, minDistance=7, blockSize=7)

while True:
    ret, frame = cap.read()
    if not ret:
        break
    #take negative
    frame = 255 - frame
    red_rect_top, red_rect_botm,direction = iterative_match(frame,left_arrow_template,right_arrow_template)
    
    x1 = red_rect_top[0]
    x2 = red_rect_botm[0]
    y1 = red_rect_top[1]
    y2 = red_rect_botm[1]

    roi = frame[y1:y2,x1:x2]
    print(direction)
    # Draw rectangles if matches are found
    if red_rect_top and red_rect_botm:
        #print("Detected")
        cv2.rectangle(frame, red_rect_top, red_rect_botm, (0, 0, 255), 5)  # Red rectangle for right arrow
    
    
    #Detect direction -----------------------------------------------------------------------------------------------
    # Preprocess the frame
    extracted_frame = roi
    preprocessed = preprocess(extracted_frame)
    
    # Detect contours
    contours, hierarchy = cv2.findContours(preprocessed, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Detect good features to track
    frame_gray = cv2.cvtColor(extracted_frame, cv2.COLOR_BGR2GRAY)
    mask = np.zeros_like(frame_gray)
    mask[:] = 255
    corners = cv2.goodFeaturesToTrack(frame_gray, mask=None, **feature_params)
#
    if corners is not None:
       corners = corners.reshape((-1, 2))

       for contour in contours:
           # Find the points on the contour that match the detected corners
           matched_points = match_points(contour, corners)

           # If a certain number of points match, classify it as a possible arrow
           if len(matched_points) > 5:  # Adjust threshold as needed
               cv2.drawContours(extracted_frame, [contour], -1, (0, 255, 0), 2)
               
               # Draw matched points
               for pt in matched_points:
                 cv2.circle(extracted_frame, (pt[0], pt[1]), 3, (0, 0, 255), -1)
               
               for cnt in [contour]:
                peri = cv2.arcLength(cnt, True)
                approx = cv2.approxPolyDP(cnt, 0.025 * peri, True)
                hull = cv2.convexHull(approx, returnPoints=False)
                sides = len(hull)
                #print(sides)
  
                if 6 > sides > 3 and sides + 2 == len(approx):
                   arrow_tip = find_tip(approx[:,0,:], hull.squeeze())
                   if arrow_tip:
                       #print(arrow_tip)
                       tail_tip = find_tail(approx,arrow_tip)
                       #cv2.drawContours(img, [cnt], -1, (0, 255, 0), 3)
                       cv2.circle(extracted_frame, arrow_tip, 3, (0, 0, 255), cv2.FILLED)
                       cv2.circle(extracted_frame,tail_tip,3,(255,0,0),cv2.FILLED)
                       #cv2.circle(extracted_frame,(tail_tip+arrow_tip)/2,3,(0,255,0),cv2.FILLED)
                       #print("Tip: ",arrow_tip," Tail: ",tail_tip)
                       midpoint_x = (arrow_tip[0] + tail_tip[0]) // 2
                       midpoint_y = (arrow_tip[1] + tail_tip[1]) // 2
                       midpoint = (midpoint_x, midpoint_y)
                       if arrow_tip[0] - tail_tip[0] > 0:
                           cv2.putText(extracted_frame, "Pointing Right", midpoint, cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2, cv2.LINE_AA)
                           #print("Pointing Right")
                       else :
                           cv2.putText(extracted_frame, "Pointing Left", midpoint, cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2, cv2.LINE_AA)
                           #print("Pointing Left")
    
    #for cnt in contours:
    #  peri = cv2.arcLength(cnt, True)
    #  approx = cv2.approxPolyDP(cnt, 0.025 * peri, True)
    #  hull = cv2.convexHull(approx, returnPoints=False)
    #  sides = len(hull)
  #
    #  if 6 > sides > 3 and sides + 2 == len(approx):
    #      arrow_tip = find_tip(approx[:,0,:], hull.squeeze())
    #      if arrow_tip:
    #          tail_tip = find_tail(approx,arrow_tip)
    #          cv2.drawContours(extracted_frame, [cnt], -1, (0, 255, 0), 3)
    #          cv2.circle(extracted_frame, arrow_tip, 3, (0, 0, 255), cv2.FILLED)
    #          cv2.circle(extracted_frame,tail_tip,3,(255,0,0),cv2.FILLED)
    #          print("Tip: ",arrow_tip," Tail: ",tail_tip)
    #          if arrow_tip[0] - tail_tip[0] > 0:
    #              cv2.putText(frame, "Pointing Right", arrow_tip, cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)
    #          else :
#cv2.putText(frame, "Pointing Left", arrow_tip, cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2, cv2.LINE_AA)


    # Display the result
    cv2.imshow("Arrow Detection",frame)

    cv2.imshow('Extracted arrow',extracted_frame)
    #cv2.imshow('Extracted Image',roi)
    if cv2.waitKey(10) & 0xFF == ord('q'):
        break


cap.release()
cv2.destroyAllWindows()


R
R
L
L
L
L
L
R
R
R
R
L
R
R
R
R
R
R
L
L
L
L
L
L
L
R
R
R
R
R
R
R
R
R
L
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
R
L
L
L
L
L
L
L
L
L
L
L
R
L
L
L
L
L
L
L
L
R
L
L
L
L
L
L
L
L
L
L
L
R
L
L
L
L
L
L
L
R
R
R
